# Machine Learning Project

## Setup

### Import

In [1]:
# Import os for operating system functions.
import os
# Import numpy for mathematical computation.
import numpy as np
# Import pandas for data manipulation.
import pandas as pd
# Import datetime for local time retrieval.
from datetime import datetime
# Import logging for file logging.
import logging
# Import time for duration measurement.
import time

# Import random, clone, make_scorer, and _fit_and_score for simulated annealing.
import random
from sklearn.base import clone
from sklearn.metrics import make_scorer
from sklearn.model_selection._validation import _fit_and_score

# Import StandardScaler for data pre-processing.
from sklearn.preprocessing import StandardScaler
# Import train_test_split for model selection.
from sklearn.model_selection import train_test_split
# Import KFold and f1_score for cross-validation.
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, classification_report

# Import SK, XGB, and CatBoost classifiers.
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

### Initialisation

In [2]:
logging.basicConfig(
	filename="main.log",
	level=logging.INFO,
	format="%(asctime)s - %(message)s",
	datefmt="%Y-%m-%d %H:%M:%S"
)

HYPERPARAMETER_ALIASES = {
	"α": "learning_rate",
	"τ": "max_iter",
	"θ": "max_leaf_nodes",
	"Δ": "max_depth",
	"l": "min_samples_leaf",
	"seed": "random_state",
	"Δ": "max_depth",
	"η": "n_estimators",
	"l2": "reg_lambda",
	"seed": "random_state",
}

MODEL_CLASSES = {
	"SKLknn": KNeighborsClassifier,
	"SKLsvm": SVC,
	"SKLrf": RandomForestClassifier,
	"SKLgb": GradientBoostingClassifier,
	"SKLhgb": HistGradientBoostingClassifier,
	"XGBgb": XGBClassifier,
	"CBgb": CatBoostClassifier
}

# Load train and test datasets.
S_train = pd.read_csv("./data/train.csv")
S_train_tfidf = pd.read_csv("./data/train_tfidf_features.csv")
S_test = pd.read_csv("./data/test.csv")
S_test_tfidf = pd.read_csv("./data/test_tfidf_features.csv")

# Extract train features, train labels, and test features.
X_train = S_train_tfidf.iloc[:, 2:].values
y_train = S_train["label"].values.reshape(-1, 1)
X_test = S_test_tfidf.iloc[:, 1:].values

## Task 1

### logreg Model

In [3]:
def σ(z): return 1 / (1 + np.exp(-z))
def bce_loss(y, ŷ): return (-1/(y.shape[0])) * np.sum(y * np.log(ŷ) + (1 - y) * np.log(1 - ŷ))

# Return dw and db, for some X, y, ŷ, w, R, and λ.
def gradients_logreg(X, y, ŷ, w, R=None, λ=0):
	m, _ = X.shape
	dw = 1/m * np.dot(X.T, (ŷ - y))
	db = 1/m * np.sum(ŷ - y)
	if R == "L2":
		dw += λ * w / m
	elif R == "L1":
		dw += λ * np.sign(w) / m
	return dw, db

# Return (w, b) from gradient descent on X_train and y_train, for some τ, α, G, β, R, and λ.
def train_logreg(X_train, y_train, τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	m, n = X_train.shape
	w, b = np.zeros((n, 1)), 0
	for epoch in range(τ):
		if G == "full-batch":
			X_batch, y_batch = X_train, y_train
			ŷ = σ(np.dot(X_batch, w) + b)
			dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
			w, b = w - α*dw, b - α*db
		elif G == "mini-batch":
			for i in range(0, m, β):
				X_batch, y_batch = X_train[i:i+β], y_train[i:i+β]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
		elif G == "stochastic":
			for i in range(m):
				X_batch, y_batch = X_train[i:i+1], y_train[i:i+1]
				ŷ = σ(np.dot(X_batch, w) + b)
				dw, db = gradients_logreg(X_batch, y_batch, ŷ, w, R, λ)
				w, b = w - α*dw, b - α*db
	return w, b

# Return array of predictions, where each prediction is 1 if corresponding ŷ entry > 0.5, and 0 otherwise.
def predict_logreg(wb_tuple, X):
	w, b = wb_tuple
	ŷ = σ(np.dot(X, w) + b)
	return np.array([1 if p > 0.5 else 0 for p in ŷ])

# Train model, make predictions, and save predictions to CSV file.
def generate_predictions_logreg(τ=1000, α=0.1, G="mini-batch", β=128, R=None, λ=0):
	start_time = time.time()
	w, b = train_logreg(np.array(X_train), np.array(y_train), τ, α, G, β, R, λ)
	predictions = predict_logreg((w, b), np.array(X_test))
	os.makedirs("./predictions/logreg/", exist_ok=True)
	file_name = os.path.join("./predictions/logreg/", f"τ={τ},α={α},G={G},β={β},R={R},λ={λ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 2

### PCA

In [4]:
def apply_pca(x):
	scaler = StandardScaler()
	X_train_scaled = scaler.fit_transform(X_train)
	X_test_scaled = scaler.transform(X_test)
	if 0 <= x <= 1:
		# x represents variance threshold.
		pca = PCA(n_components=None)
		pca.fit(X_train_scaled)
		c = np.argmax(np.cumsum(pca.explained_variance_ratio_) >= x) + 1
		v = x
	else:
		# x represents number of components.
		pca = PCA(n_components=x)
		pca.fit(X_train_scaled)
		c = x
		v = sum(pca.explained_variance_ratio_)
	# Transform train and test datasets.
	X_train_pca = pca.transform(X_train_scaled)
	X_test_pca = pca.transform(X_test_scaled)
	return X_train_pca, X_test_pca, c, v

### SKLknn Model

In [5]:
def train_SKLknn(X_train, y_train, k=5, W="uniform", p=2, m="minkowski"):
	model = KNeighborsClassifier(
		n_neighbors=k,
		weights=W,
		p=p,
		metric=m,
		n_jobs=-1
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLknn(model, X): return model.predict(X)

def generate_predictions_SKLknn(k=5, W="uniform", p=2, m="minkowski"):
	start_time = time.time()
	model = train_SKLknn(np.array(X_train), np.array(y_train), k, W, p, m)
	predictions = predict_SKLknn(model, np.array(X_test))
	os.makedirs("./predictions/SKLknn/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLknn/", f"k={k},W={W},p={p},m={m}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### Combination

In [6]:
# Train model, make model predictions, and save model predictions to CSV file.
def generate_predictions_pcaknn(x):
	start_time = time.time()
	X_train_pca, X_test_pca, c, v = apply_pca(x)
	model = train_SKLknn(X_train_pca, y_train, k=2)
	predictions = predict_SKLknn(model, X_test_pca)
	os.makedirs("./predictions/pcaknn/", exist_ok=True)
	file_name = os.path.join("./predictions/pcaknn/", f"pcaknn(ρ={c},ν={v:.2f}).csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

## Task 3: Other Models

### SKLlogreg Model (WIP)

### SKLrf Model

In [7]:
def train_SKLrf(X_train, y_train, η=100, cri="gini", Δ=None, ψ=2, l=1, θ=None, seed=None):
	model = RandomForestClassifier(
		n_estimators=η,
		criterion=cri,
		max_depth=Δ,
		min_samples_split=ψ,
		min_samples_leaf=l,
		max_leaf_nodes=θ,
		n_jobs=-1,
		random_state=seed
	)
	model.fit(X_train, y_train.ravel())
	return model

def predict_SKLrf(model, X): return model.predict(X)

def generate_predictions_SKLrf(η=100, cri="gini", Δ=None, ψ=2, l=1, θ=None, seed=None):
	start_time = time.time()
	model = train_SKLrf(np.array(X_train), np.array(y_train), η, cri, Δ, ψ, l, θ, seed)
	predictions = predict_SKLrf(model, np.array(X_test))
	os.makedirs("./predictions/SKLrf/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLrf/", f"η={η},cri={cri},Δ={Δ},ψ={ψ},l={l},θ={θ},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLsvm Model

In [8]:
# Train model, make predictions, and save predictions to CSV file.
def train_SKLsvm(X_train, y_train, C=1.0, ker="rbf", d=3, γ="scale", τ=-1, seed=None):
	model = SVC(
		C=C,
		kernel=ker,
		degree=d,
		gamma=γ,
		max_iter=τ,
		random_state=seed
	)
	model.fit(X_train, y_train.ravel())
	return model

def predict_SKLsvm(model, X): return model.predict(X)

def generate_predictions_SKLsvm(C=1.0, ker="rbf", d=3, γ="scale", τ=-1, seed=None):
	start_time = time.time()
	model = train_SKLsvm(np.array(X_train), np.array(y_train), C, ker, d, γ, τ, seed)
	predictions = predict_SKLsvm(model, np.array(X_test))
	os.makedirs("./predictions/SKLsvm/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLsvm/", f"C={C},ker={K},d={d},γ={γ},τ={τ},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLgb Model

In [9]:
def train_SKLgb(X_train, y_train, α=0.1, η=100, ss=1.0, ψ=2, l=1, Δ=3, seed=None, θ=None):
	model = GradientBoostingClassifier(
		learning_rate=α,
		n_estimators=η,
		subsample=ss,
		min_samples_split=ψ,
		min_samples_leaf=l,
		max_depth=Δ,
		random_state=seed,
		max_leaf_nodes=θ
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLgb(model, X): return model.predict(X)

def generate_predictions_SKLgb(α=0.1, η=100, ss=1.0, ψ=2, l=1, Δ=3, seed=None, θ=None):
	start_time = time.time()
	model = train_SKLgb(np.array(X_train), np.array(y_train), α, η, ss, ψ, l, Δ, seed, θ)
	predictions = predict_SKLgb(model, np.array(X_test))
	os.makedirs("./predictions/SKLgb/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLgb/", f"α={α},η={η},ss={ss},ψ={ψ},l={l},Δ={Δ},seed={seed},θ={θ}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### SKLhgb Model

In [10]:
def train_SKLhgb(X_train, y_train, α=0.1, τ=100, θ=31, Δ=None, l=20, seed=None):
	model = HistGradientBoostingClassifier(
		learning_rate=α,
		max_iter=τ,
		max_leaf_nodes=θ,
		max_depth=Δ,
		min_samples_leaf=l,
		random_state=seed
	)
	model.fit(X_train, y_train)
	return model

def predict_SKLhgb(model, X): return model.predict(X)

def generate_predictions_SKLhgb(α=0.1, τ=100, θ=31, Δ=None, l=20, seed=None):
	start_time = time.time()
	model = train_SKLhgb(np.array(X_train), np.array(y_train), α, τ, θ, Δ, l, seed)
	predictions = predict_SKLhgb(model, np.array(X_test))
	os.makedirs("./predictions/SKLhgb/", exist_ok=True)
	file_name = os.path.join("./predictions/SKLhgb/", f"α={α},τ={τ},θ={θ},Δ={Δ},l={l},seed={seed}.csv")
	submission = pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### XGBgb Model

In [11]:
def train_XGBgb(X_train, y_train, η=100, Δ=3, Θ=0, α=0.1, B="gbtree", ss=1, l1=0, l2=1, seed=None):
	xgb_model = XGBClassifier(
		n_estimators=η,
		max_depth=Δ,
		max_leaves=Θ if Θ > 0 else None,
		learning_rate=α,
		objective="binary:logistic",
		booster=B,
		subsample=ss,
		reg_alpha=l1,
		reg_lambda=l2,
		random_state=seed,
		n_jobs=-1
	)
	xgb_model.fit(X_train, y_train.ravel())
	return xgb_model

def predict_XGBgb(model, X):
	return model.predict(X)

def generate_predictions_XGBgb(η=100, Δ=3, Θ=0, α=0.1, B="gbtree", ss=1, l1=0, l2=1, seed=None):
	start_time = time.time()
	xgb_model = train_XGBgb(np.array(X_train), np.array(y_train), η, Δ, Θ, α, B, ss, l1, l2, seed)
	predictions = predict_XGBgb(xgb_model, np.array(X_test))
	os.makedirs("./predictions/XGBgb/", exist_ok=True)
	file_name = os.path.join("./predictions/XGBgb/", f"η={η},Δ={Δ},Θ={Θ},α={α},B={B},ss={ss},l1={l1},l2={l2},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

### CBgb Model

In [12]:
def train_CBgb(X_train, y_train, α=None, ss=None, Δ=None, η=None, l2=None, seed=None):
	model = CatBoostClassifier(
		learning_rate=α,
		subsample=ss,
		max_depth=Δ,
		n_estimators=η,
		reg_lambda=l2,
		random_state=seed
	)
	model.fit(X_train, y_train)
	return model

def predict_CBgb(model, X): return model.predict(X)

def generate_predictions_CBgb(α=None, ss=None, Δ=None, η=None, l2=None, seed=None):
	start_time = time.time()
	model = train_CBgb(np.array(X_train), np.array(y_train), α, ss, Δ, η, l2, seed)
	predictions = predict_CBgb(model, np.array(X_test))
	os.makedirs("./predictions/CBgb/", exist_ok=True)
	file_name = os.path.join("./predictions/CBgb/", f"α={α},ss={ss},Δ={Δ},η={η},l2={l2},seed={seed}.csv")
	pd.DataFrame({"id": S_test["id"], "label": predictions}).to_csv(file_name, index=False)
	end_time = time.time()
	logging.info(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")
	print(f"Predictions file {file_name} generated in {end_time - start_time:.2f}s.")

generate_predictions_CBgb()

Learning rate set to 0.0347
0:	learn: 0.6885509	total: 90.5ms	remaining: 1m 30s
1:	learn: 0.6847681	total: 113ms	remaining: 56.5s
2:	learn: 0.6808560	total: 138ms	remaining: 45.7s
3:	learn: 0.6774119	total: 160ms	remaining: 39.7s
4:	learn: 0.6745607	total: 179ms	remaining: 35.6s
5:	learn: 0.6716783	total: 200ms	remaining: 33.1s
6:	learn: 0.6689180	total: 219ms	remaining: 31s
7:	learn: 0.6666538	total: 239ms	remaining: 29.6s
8:	learn: 0.6648337	total: 257ms	remaining: 28.3s
9:	learn: 0.6627754	total: 277ms	remaining: 27.4s
10:	learn: 0.6609090	total: 297ms	remaining: 26.7s
11:	learn: 0.6587380	total: 318ms	remaining: 26.2s
12:	learn: 0.6566636	total: 337ms	remaining: 25.6s
13:	learn: 0.6547374	total: 356ms	remaining: 25.1s
14:	learn: 0.6530832	total: 375ms	remaining: 24.6s
15:	learn: 0.6516802	total: 394ms	remaining: 24.2s
16:	learn: 0.6499572	total: 412ms	remaining: 23.8s
17:	learn: 0.6481825	total: 431ms	remaining: 23.5s
18:	learn: 0.6467947	total: 449ms	remaining: 23.2s
19:	learn: 0.

### Cross-Validation Area

In [13]:
def crossvalidate_grid(train_fn, predict_fn, grid, X, y, k=2):
	kf = KFold(n_splits=k, shuffle=True)
	best_f1 = -np.inf
	best_hyperparameters = None
	for hyperparameters in grid:
		f1_scores = []
		for i_train, i_val in kf.split(X):
			X_train, X_val = X[i_train], X[i_val]
			y_train, y_val = y[i_train], y[i_val]
			model = train_fn(X_train, y_train, **hyperparameters)
			y_pred = predict_fn(model, X_val)
			f1_scores.append(f1_score(y_val, y_pred, average="macro"))
		mean_f1 = np.mean(f1_scores)
		if mean_f1 > best_f1:
			best_f1 = mean_f1
			best_hyperparameters = hyperparameters
	print(f"Best hyperparameters in grid:", best_hyperparameters)
	print(f"Best {k}-fold cross-validation f1 score:", best_f1)
	return best_hyperparameters, best_f1

# def crossvalidate_and_generate_predictions_grid(model_name, grid, k=5):
# 	train_fn = globals().get(f"train_{model_name.lower()}")
# 	predict_fn = globals().get(f"predict_{model_name.lower()}")
# 	generate_predictions_fn = globals().get(f"generate_predictions_{model_name.lower()}")
# 	if not train_fn or not predict_fn or not generate_predictions_fn: raise ValueError(f"Expected train_{model_name} or predict_{model_name} to exist.")
# 	best_hyperparameters, _ = crossvalidate_grid(train_fn, predict_fn, grid, X_train, y_train, k=k)
# 	generate_predictions_fn(**best_hyperparameters)

In [14]:
# grid = [
# 	{"τ": 10000, "α": 0.0825, "G": "mini-batch", "β": 128, "R": "L2", "λ": 1},
# 	{"τ": 10000, "α": 0.085, "G": "mini-batch", "β": 128, "R": "L2", "λ": 1}
# ]
# crossvalidate_and_generate_predictions("logreg", grid, k=2)

# grid = [
# 	{"η": 100, "C": "gini", "Δ": None, "ψ": 2, "l": 1, "θ": None, "seed": None},
# 	{"η": 200, "C": "entropy", "Δ": 10, "ψ": 3, "l": 2, "θ": 50, "seed": 42}
# ]
# crossvalidate_and_generate_predictions("SKLrf", grid, k=5)

In [15]:
class SA:
	def __init__(self, estimator, grid, scoring=f1_score, T=10, T_min=0.001, α=0.9, n_trans=5, max_iter=100, max_runtime=60, cv=5, n_jobs=-1, max_f1=np.inf, min_improvement=1e-4, patience=10):
		self.estimator = estimator
		self.grid = grid
		self.scoring = scoring
		self.T = T
		self.T_min = T_min
		self.α = α
		self.n_trans = n_trans
		self.max_iter = max_iter
		self.max_runtime = max_runtime
		self.cv = cv
		self.n_jobs = n_jobs
		self.max_f1 = max_f1
		self.min_improvement = min_improvement
		self.patience = patience

		self.best_hyperparameters_ = None
		self.best_f1_ = None
		self.grid_f1s_ = None
		self.runtime_ = None
		self._set_dynamic_params()

	def _set_dynamic_params(self):
		num_hyperparameters = np.prod([len(v) for v in self.grid.values() if isinstance(v, list)])
		self.T = 10 * num_hyperparameters
		self.T_min = 0.01 * self.T
		self.α = 0.9

	def _accept_prob(self, old_f1, new_f1, T):
		T += 0.01
		return np.exp((new_f1 - old_f1) / T)

	def _dt(self, t0, t1): return t1 - t0 if t0 is not None else 0

	def fit(self, X, y):
		if isinstance(X, pd.DataFrame):
			X = X.to_numpy()
		if isinstance(y, pd.DataFrame):
			y = y.to_numpy()
		elif isinstance(y, (list, pd.Series)):
			y = np.array(y)
		T = self.T
		T_min = self.T_min
		α = self.α
		max_iter = self.max_iter
		n_trans = self.n_trans
		grid = self.grid
		max_runtime = self.max_runtime
		cv = self.cv
		old_hyperparameters = {k: np.random.choice(v) if isinstance(v, list) else np.random.uniform(v[0], v[1]) for k, v in grid.items()}
		old_est = clone(self.estimator)
		old_est.set_params(**old_hyperparameters)
		old_f1, old_std = self._evaluate_f1(old_est, X, y, cv)
		best_f1 = old_f1
		best_hyperparameters = old_hyperparameters
		states_checked = {tuple(sorted(old_hyperparameters.items())): (old_f1, old_std)}
		total_iter = 1
		grid_f1s = [(1, T, old_f1, old_std, old_hyperparameters)]
		time_at_start = time.time()
		t_elapsed = self._dt(time_at_start, time.time())
		no_improvement_count = 0
		while T > T_min and total_iter < max_iter and t_elapsed < max_runtime and best_f1 < self.max_f1:
			for _ in range(self.n_trans):
				new_hyperparameters = self._generate_new_hyperparameters(old_hyperparameters, grid)
				new_f1, new_std = self._evaluate_f1_for_hyperparameters(new_hyperparameters, X, y, cv, states_checked)
				if new_f1 >= self.max_f1: break
				grid_f1s.append((total_iter, T, new_f1, new_std, new_hyperparameters))
				if new_f1 > best_f1:
					best_f1 = new_f1
					best_hyperparameters = new_hyperparameters
					no_improvement_count = 0
				else:
					no_improvement_count += 1
				print(f"{total_iter} T: {T:.5f}, f1: {new_f1:.6f}, std: {new_std:.6f}, hyperparameters: {new_hyperparameters}")
				if self._accept_prob(old_f1, new_f1, T) > np.random.random():
					old_hyperparameters = new_hyperparameters
					old_f1 = new_f1
				t_elapsed = self._dt(time_at_start, time.time())
				total_iter += 1
			if new_f1 >= self.max_f1:
				print(f"Max f1 reached {new_f1}!")
				break
			if no_improvement_count >= self.patience and T <= self.T_min:
				print("Early stopping due to lack of improvement.")
				break
			T *= α
		self.runtime_ = t_elapsed
		self.grid_f1s_ = grid_f1s
		self.best_f1_ = best_f1
		self.best_hyperparameters_ = best_hyperparameters

	def _generate_new_hyperparameters(self, old_hyperparameters, grid):
		new_hyperparameters = old_hyperparameters.copy()
		rand_key = np.random.choice(list(grid.keys()))
		val = grid[rand_key]
		if isinstance(val, list):
			sample_space = [v for v in val if v != old_hyperparameters[rand_key]]
			new_hyperparameters[rand_key] = np.random.choice(sample_space) if sample_space else np.random.choice(val)
		elif isinstance(val, tuple) and len(val) == 2:
			new_hyperparameters[rand_key] = np.random.uniform(val[0], val[1])
		return new_hyperparameters

	def _evaluate_f1_for_hyperparameters(self, hyperparameters, X, y, cv, states_checked):
		hyperparameters_tuple = tuple(sorted(hyperparameters.items()))
		if hyperparameters_tuple in states_checked:
			return states_checked[hyperparameters_tuple]
		else:
			est = clone(self.estimator)
			est.set_params(**hyperparameters)
			f1, std = self._evaluate_f1(est, X, y, cv)
			states_checked[hyperparameters_tuple] = (f1, std)
			return f1, std

	def _evaluate_f1(self, estimator, X, y, cv):
		if self.n_jobs > 1:
			out = Parallel(n_jobs=self.n_jobs)(
				delayed(_fit_and_score)(clone(estimator), X, y, self.scoring, train, test, verbose=True,
				                        parameters={}, fit_params={}, return_parameters=False, error_score='raise')
				for train, test in KFold(cv).split(X)
			)
		else:
			scores = []
			for train, test in KFold(cv).split(X):
				estimator.fit(X[train], y[train])
				y_pred = estimator.predict(X[test])
				scores.append(self.scoring(y[test], y_pred))
			out = (np.mean(scores), np.std(scores))
		return out

def crossvalidate_sa(model_name, grid, T=10, T_min=0.001, α=0.9, n_trans=5, max_iter=100, max_runtime=60, cv=5, min_improvement=1e-4, patience=10):
	grid_mapped = {HYPERPARAMETER_ALIASES.get(k, k): v for k, v in grid.items()}
	model_class = MODEL_CLASSES.get(model_name)
	if model_class is None: raise ValueError(f"Model '{model_name}' is not recognised.")
	model = model_class()
	sa = SA(
		estimator=model,
		grid=grid_mapped,
		scoring=f1_score,
		T=T,
		T_min=T_min,
		α=α,
		n_trans=n_trans,
		max_iter=max_iter,
		max_runtime=max_runtime,
		cv=cv,
		min_improvement=min_improvement,
		patience=patience
	)
	sa.fit(X_train, y_train)
	return sa.best_f1_, sa.best_hyperparameters_, sa.grid_f1s_, sa.runtime_

In [16]:
# t=None, α=None, δ=None, l2l=None, ss=None, Δ=None, η=None, l2=None, seed=None
grid = {
	"α": (0.06, 0.12),
	"Δ": [12],
	"η": [100],
	"l2": (0.1, 1),
}
best_f1, best_hyperparameters, grid_f1s, runtime = crossvalidate_sa(model_name="CBgb", grid=grid)
print(best_f1)
print(best_hyperparameters)

0:	learn: 0.6820093	total: 808ms	remaining: 1m 20s
1:	learn: 0.6733821	total: 1.66s	remaining: 1m 21s
2:	learn: 0.6635628	total: 2.41s	remaining: 1m 17s
3:	learn: 0.6567553	total: 3.29s	remaining: 1m 18s
4:	learn: 0.6507293	total: 4.07s	remaining: 1m 17s
5:	learn: 0.6443502	total: 4.93s	remaining: 1m 17s
6:	learn: 0.6407615	total: 5.87s	remaining: 1m 18s
7:	learn: 0.6369212	total: 6.62s	remaining: 1m 16s
8:	learn: 0.6317432	total: 7.39s	remaining: 1m 14s
9:	learn: 0.6288005	total: 8.18s	remaining: 1m 13s
10:	learn: 0.6248830	total: 8.99s	remaining: 1m 12s
11:	learn: 0.6215726	total: 9.76s	remaining: 1m 11s
12:	learn: 0.6177793	total: 10.6s	remaining: 1m 11s
13:	learn: 0.6156545	total: 11.4s	remaining: 1m 10s
14:	learn: 0.6132241	total: 12.2s	remaining: 1m 9s
15:	learn: 0.6106156	total: 13.2s	remaining: 1m 9s
16:	learn: 0.6086867	total: 13.9s	remaining: 1m 8s
17:	learn: 0.6064976	total: 14.7s	remaining: 1m 7s
18:	learn: 0.6040060	total: 15.5s	remaining: 1m 6s
19:	learn: 0.6021503	total: